In [0]:
import matplotlib.pyplot as plt
import numpy as np
%tensorflow_version 2.x
import tensorflow as tf
import os

In [49]:
!wget 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt'

--2020-05-07 16:50:20--  https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.204.128, 2404:6800:4008:c04::80
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.204.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: ‘shakespeare.txt’

shakespeare.txt     100%[===================>]   1.06M  --.-KB/s    in 0.01s   

2020-05-07 16:50:20 (88.0 MB/s) - ‘shakespeare.txt’ saved [1115394/1115394]



In [0]:
text = open('shakespeare.txt').read()
print(text[:250])

## Process the dataset

In [0]:
vocab = sorted(set(text))    #Contains every character used

In [0]:
char2index = {char: index for index, char in enumerate(vocab)}   #character to index
index2char = np.array(vocab)        #index to character

In [0]:
text_as_int = np.array([char2index[char] for char in text])      #Converting the `text` into `int`

In [0]:
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)       #Creating tensors

In [0]:
#Creating batches of data
char_dataset = char_dataset.batch(100 + 1, drop_remainder=True) #lenght of the each sentence=100 

In [0]:
def inp_target(sent):
    inp = sent[:-1]
    target = sent[1:]
    return inp, target

In [0]:
dataset = char_dataset.map(inp_target)   #Prepare input and target data

In [0]:
BATCH_SIZE = 64
BUFFER_SIZE = 10000

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

In [0]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
    model = tf.keras.models.Sequential([tf.keras.layers.Embedding(input_dim=vocab_size,
                                                                  output_dim=embedding_dim,
                                                                  batch_input_shape=[batch_size, None]),
                                        
                                        tf.keras.layers.LSTM(units=rnn_units,
                                                             return_sequences=True,
                                                             stateful=True,
                                                             recurrent_initializer=tf.keras.initializers.GlorotNormal()),
                                        
                                        tf.keras.layers.Dense(vocab_size)
    ])

    return model

In [0]:
vocab_size = len(vocab)
embedding_dim = 256
rnn_units = 1024

In [0]:
model = build_model(vocab_size, embedding_dim, rnn_units, BATCH_SIZE)

In [22]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (64, None, 256)           16640     
_________________________________________________________________
lstm (LSTM)                  (64, None, 1024)          5246976   
_________________________________________________________________
dense (Dense)                (64, None, 65)            66625     
Total params: 5,330,241
Trainable params: 5,330,241
Non-trainable params: 0
_________________________________________________________________


In [0]:
adam_optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
model.compile(
    optimizer=adam_optimizer,
    loss=loss
)

In [0]:
checkpoint_dir = 'checkpoints'
os.makedirs('checkpoint_dir', exist_ok=True)

checkpoint_prefix = os.path.join(checkpoint_dir, 'ckpt_{epoch}')

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True
)

### Execute the training

In [0]:
EPOCHS=40

In [28]:
history = model.fit(x=dataset,
                    epochs=EPOCHS,
                    callbacks=[checkpoint_callback])

Epoch 1/40
172/172 [==============================] - 28s 161ms/step - loss: 2.5909
Epoch 2/40
172/172 [==============================] - 28s 162ms/step - loss: 1.8888
Epoch 3/40
172/172 [==============================] - 28s 163ms/step - loss: 1.6382
Epoch 4/40
172/172 [==============================] - 28s 163ms/step - loss: 1.5048
Epoch 5/40
172/172 [==============================] - 28s 163ms/step - loss: 1.4243
Epoch 6/40
172/172 [==============================] - 28s 163ms/step - loss: 1.3675
Epoch 7/40
172/172 [==============================] - 28s 163ms/step - loss: 1.3226
Epoch 8/40
172/172 [==============================] - 28s 163ms/step - loss: 1.2843
Epoch 9/40
172/172 [==============================] - 28s 163ms/step - loss: 1.2482
Epoch 10/40
172/172 [==============================] - 28s 163ms/step - loss: 1.2136
Epoch 11/40
172/172 [==============================] - 28s 164ms/step - loss: 1.1780
Epoch 12/40
172/172 [==============================] - 28s 163ms/step - lo

## Text Generation


In [0]:
simplified_batch_size = 1

model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)

model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))

model.build(tf.TensorShape([simplified_batch_size, None]))

In [34]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (1, None, 256)            16640     
_________________________________________________________________
lstm_2 (LSTM)                (1, None, 1024)           5246976   
_________________________________________________________________
dense_2 (Dense)              (1, None, 65)             66625     
Total params: 5,330,241
Trainable params: 5,330,241
Non-trainable params: 0
_________________________________________________________________


In [0]:
def generate_text(model, start_string, num_generate = 1000, temperature=1.0):
    
    input_indices = [char2index[s] for s in start_string]
    input_indices = tf.expand_dims(input_indices, 0)

    text_generated = []
    
    for char_index in range(num_generate):
        predictions = model(input_indices)
        predictions = tf.squeeze(predictions, 0)

        predictions = predictions / temperature
        predicted_id = tf.random.categorical(predictions,
                                             num_samples=1)[-1,0].numpy()

        input_indices = tf.expand_dims([predicted_id], 0)

        text_generated.append(index2char[predicted_id])

    return (start_string + ''.join(text_generated))

In [61]:
print(generate_text(model, start_string=u"ROMEO: "))

ROMEO: farewell; for I was time to be perfidiously;
Meaning the tide with you.
For I were not weep we are after tow.

GLOUCESTER:

KING EDWARD IV:
Clarence and Somerset both grow! a malice,
That thieves do so, so that the kingly hath
Found his most sovereign? O my well, come for't.

PETRICANI:
Go, away with her, trust me with roaring more requite; like daughter,
Nor more such name in this exgredier.
What looks the tribunes of the news I turn me from the mere you:
From whence shall I be patient. You, my lords,
With thoughts from birth of all my loving father.

KING HENRY VI:
We does with you; and so he shall never
Be calm the Duke of Lancaster letting
Than all worm of inecity,
Which wither'd his friends, he would desire us:
I have seen an inthrone can be too: true, instation
That by my faith dread father: Mont go what:
I'll not be longer than they rejeight,
Which makes thee appear in the midnight
For peoplext here with her that shall not prove,
And I'll believe me to my nn occasion of m

In [45]:
print(generate_text(model, start_string=u"ROMEO: ", temperature=0.01))

ROMEO: IIGHAM:
Marry, my lord, let me know your miserable.
And it concluded the prince, craftress lies
Against the words of slaughing men!

Second Servant:
O, this is it that makes this spoil were frown.

LUCENTIO:
I pray you are to make thee speak.

HENRY BOLINGBROKE:
Many years of happy son, he stands by what thou wert possess'd,
Which art possess'd now to depose thyself.
Why, cousin, I'll keep the prince, and call thee back
With twenty hundred thousand times before the water.

GLOUCESTER:
Why, what is that to me that?

GLOUCESTER:
I cannot blame her: by God's holy mother,
She hath had more for fathers you shall be the man.
But stay the triumph of great Bolingbroke?
Gardener, for telling me these news of his transported
and hanging of a king: beseech you, and my knightly time
To say 'What is it, Buckingham?

BUCKINGHAM:
Marry, my lord, lett she shall bring forth my heart,
Which is the bell: so sighs and tears and groans
Show minutes, time, go with me;
I charge thee in the victory of 

## Save the model

In [0]:
model_name = 'shakespeare_text_generation.h5'
model.save(model_name, save_format='h5')